In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tw
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/256x256-images/train.zip
/kaggle/input/256x256-images/masks.zip
/kaggle/input/256x256-images/__results__.html
/kaggle/input/256x256-images/__notebook__.ipynb
/kaggle/input/256x256-images/__output__.json
/kaggle/input/256x256-images/custom.css
/kaggle/input/256x256-images/__results___files/__results___6_0.png
/kaggle/input/image-recognition-gender-detection-inceptionv3/__results__.html
/kaggle/input/image-recognition-gender-detection-inceptionv3/__output__.json
/kaggle/input/image-recognition-gender-detection-inceptionv3/weights.best.inc.male.hdf5
/kaggle/input/image-recognition-gender-detection-inceptionv3/custom.css
/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___40_0.png
/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___11_1.png
/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___23_0.png
/kaggle/input/image-recognition-gender-detection-inceptionv3/

In [2]:
monthlySaleNumber = [25, 63, -185, 35, 67, 42,92, -35, 63, 56]

In [3]:
twDatsts = tw.data.Dataset.from_tensor_slices(monthlySaleNumber)
twDatsts

2023-01-13 20:30:37.614756: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<TensorSliceDataset shapes: (), types: tf.int32>

In [4]:
for sales in twDatsts:
    print(sales)

tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(63, shape=(), dtype=int32)
tf.Tensor(-185, shape=(), dtype=int32)
tf.Tensor(35, shape=(), dtype=int32)
tf.Tensor(67, shape=(), dtype=int32)
tf.Tensor(42, shape=(), dtype=int32)
tf.Tensor(92, shape=(), dtype=int32)
tf.Tensor(-35, shape=(), dtype=int32)
tf.Tensor(63, shape=(), dtype=int32)
tf.Tensor(56, shape=(), dtype=int32)


In [5]:
for sales in twDatsts:
    print(sales.numpy())

25
63
-185
35
67
42
92
-35
63
56


In [6]:
for sales in twDatsts.as_numpy_iterator():
    print(sales)

25
63
-185
35
67
42
92
-35
63
56


In [7]:
for sales in twDatsts.take(5):
    print(sales.numpy())

25
63
-185
35
67


In [8]:
twDatsts = twDatsts.filter(lambda x: x>0)
for sales in twDatsts.as_numpy_iterator():
    print(sales)

25
63
35
67
42
92
63
56


2023-01-13 20:30:37.720724: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [9]:
twDatsts = twDatsts.map(lambda x: x*86)
for sales in twDatsts.as_numpy_iterator():
    print(sales)

2150
5418
3010
5762
3612
7912
5418
4816


In [10]:
twDatsts = twDatsts.shuffle(5)
for sales in twDatsts.as_numpy_iterator():
    print(sales)

2150
5418
3010
3612
4816
5418
5762
7912


In [11]:
for sales_batch in twDatsts.batch(4):
    print(sales_batch.numpy()) 

[5418 2150 5762 7912]
[5418 3010 3612 4816]


In [12]:
twDatsts = tw.data.Dataset.from_tensor_slices(monthlySaleNumber)
twDatsts = twDatsts.filter(lambda x: x>0).map(lambda y: y*86).shuffle(2).batch(2)
for sales in twDatsts.as_numpy_iterator():
    print(sales)

[5418 3010]
[2150 3612]
[5762 5418]
[7912 4816]


In [13]:
imagesdst = tw.data.Dataset.list_files('/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/*', shuffle = False)
for file in imagesdst.take(5):
    print(file.numpy())

b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___11_1.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___13_0.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___23_0.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___40_0.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___41_0.png'


In [14]:
imagesdst = imagesdst.shuffle(200)
for file in imagesdst.take(10):
    print(file.numpy())

b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___40_0.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___23_0.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___11_1.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___41_0.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___13_0.png'


In [15]:
className = ['Livingthing', 'NonLivingthing']

In [16]:
imgCount = len(imagesdst)
imgCount

5

In [17]:
trainSize = int(imgCount * 0.8)
trainDS = imagesdst.take(trainSize)
testDS = imagesdst.skip(trainSize)

In [18]:
len(trainDS), len(testDS)

(4, 1)

In [19]:
p = '/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___11_1.png'
p.split("/")[-2]

'__results___files'

In [20]:
def get_label(file_path):
    import os
    return tw.strings.split(file_path, os.path.sep)[-2]

def process_image(file_path):
    label = get_label(file_path)
    img = tw.io.read_file(file_path)
    img = tw.image.decode_jpeg(img)
    img = tw.image.resize(img, [128, 128])
    return img, label

In [21]:
for t in trainDS.take(4):
    print(t.numpy())

b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___13_0.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___23_0.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___41_0.png'
b'/kaggle/input/image-recognition-gender-detection-inceptionv3/__results___files/__results___11_1.png'


In [22]:
trainDS = trainDS.map(process_image)
for img, label in trainDS.take(5):
    print("Image:", img)
    print("Label:", label)

Image: tf.Tensor(
[[[255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  ...
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]]

 [[255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  ...
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]]

 [[255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  ...
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]]

 ...

 [[255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  ...
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]]

 [[255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  ...
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]]

 [[255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  ...
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]
  [255. 255. 255.   0.]]], shape=(128, 128, 4), dtype=float32)
Label: tf.Tensor(b'__result

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


In [23]:
def scale (image, label):
    return image/255, label

trainDS = trainDS.map(scale)
for image, label in trainDS.take(5):
    print("Image:", img.numpy()[0][0])
    print("Label:", label.numpy())

Image: [255. 255. 255.   0.]
Label: b'__results___files'
Image: [255. 255. 255.   0.]
Label: b'__results___files'
Image: [255. 255. 255.   0.]
Label: b'__results___files'
Image: [255. 255. 255.   0.]
Label: b'__results___files'


Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
